In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Check Data") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [2]:
schema = StructType([
    StructField("event_time", TimestampType(), nullable=False),
    StructField("event_type", StringType(), nullable=False),
    StructField("product_id", StringType(), nullable=False),
    StructField("category_id", StringType(), nullable=False),
    StructField("category_code", StringType(), nullable=True),
    StructField("brand", StringType(), nullable=True),
    StructField("price", DoubleType(), nullable=False),
    StructField("user_id", StringType(), nullable=False),
    StructField("user_session", StringType(), nullable=False),
])


In [3]:

path = "./data/2020-Jan.csv"

df = spark.read.format("csv").option("header", "true").schema(schema).load(path)

df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



In [4]:
df_cleaned = df.dropDuplicates() \
        .dropna(subset=["event_time", "brand", "product_id", "category_id", "price", "user_id", "user_session", "category_code"]) \
        .withColumn("category_level_1", split(col("category_code"), "\\.")[0]) \
        .withColumn("category_level_2", split(col("category_code"), "\\.")[1]) \
        .withColumn("category_level_3", split(col("category_code"), "\\.")[2]) \
        .withColumn("category_level_4", split(col("category_code"), "\\.")[3]) \
        .drop("category_code") \
        .withColumn("year", year("event_time")) \
        .withColumn("month", month("event_time")) \
        .withColumn("day", day("event_time"))

In [5]:
df_cleaned.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)
 |-- category_level_1: string (nullable = true)
 |-- category_level_2: string (nullable = true)
 |-- category_level_3: string (nullable = true)
 |-- category_level_4: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [21]:
df_cleaned.select("product_id", "category_id").distinct().groupBy("product_id").count().orderBy("count", ascending=False).show()

+----------+-----+
|product_id|count|
+----------+-----+
|   3701313|    1|
| 100038888|    1|
|  10800206|    1|
|   8700495|    1|
|   4802333|    1|
| 100003332|    1|
|  28401278|    1|
|  31700044|    1|
|  16700401|    1|
|   9200351|    1|
|   4200507|    1|
|  21404126|    1|
|  18500057|    1|
|   4803400|    1|
|  50300571|    1|
|  43500040|    1|
|   1005263|    1|
|   1801987|    1|
|   1801967|    1|
|  14300042|    1|
+----------+-----+
only showing top 20 rows



In [22]:
from pyspark.sql import Window
dim_product = df_cleaned.select(
    "product_id", "category_id", "brand", "price"
).distinct().groupBy("product_id", "category_id", "brand").agg(
    round(avg("price"), 2).alias("price")
).distinct()


window = Window.partitionBy("product_id", "category_id").orderBy("brand")
dim_product = dim_product.withColumn("row_number", row_number().over(window)) \
    .filter("row_number = 1")\
    .drop("row_number")\
    .distinct()

dim_product.show()

+----------+-------------------+---------+------+
|product_id|        category_id|    brand| price|
+----------+-------------------+---------+------+
| 100000020|2232732065311228801|  kroskaa| 20.12|
| 100000024|2232732105123562109|  ubisoft| 54.04|
| 100000025|2053013553375346967| spalenka| 16.49|
| 100000040|2053013553375346967|   adamas| 19.05|
| 100000050|2232732081585127530|    delta|119.44|
| 100000070|2232732081585127530|    delta|105.79|
| 100000071|2232732081585127530|    delta| 109.4|
| 100000097|2232732081585127530|    delta|158.31|
| 100000112|2053013563693335403|    delta| 96.01|
| 100000136|2053013556990837237|     emsa| 12.44|
| 100000150|2053013556990837237|  bohemia| 28.87|
| 100000151|2053013565782098913|  respect| 71.82|
| 100000158|2053013556990837237|  bohemia| 26.16|
| 100000210|2053013554415534427|      jvc|334.63|
| 100000215|2053013555573162395|milavitsa| 23.59|
| 100000216|2053013556462354899|      dam| 48.52|
| 100000221|2053013553375346967|belashoff| 23.56|


In [17]:
dim_product.groupBy("product_id").agg(
    count("brand").alias("count brand")
).orderBy("count brand", ascending=False).show()

+----------+-----------+
|product_id|count brand|
+----------+-----------+
| 100000002|          1|
| 100000000|          1|
| 100000009|          1|
| 100000001|          1|
| 100000003|          1|
| 100000010|          1|
| 100000024|          1|
| 100000013|          1|
| 100000019|          1|
| 100000020|          1|
| 100000046|          1|
| 100000022|          1|
| 100000025|          1|
| 100000026|          1|
| 100000056|          1|
| 100000027|          1|
| 100000038|          1|
| 100000028|          1|
| 100000078|          1|
| 100000031|          1|
+----------+-----------+
only showing top 20 rows



In [11]:
dim_product.filter(col("product_id") == "17600723").distinct().show()

+----------+---------+-----+
|product_id|    brand|price|
+----------+---------+-----+
|  17600723|   doliva|19.78|
|  17600723|christina|19.78|
+----------+---------+-----+



In [28]:
dim_date = df.select(
    to_date(col("event_time")).alias("date_id"),
    month("event_time").alias("month"),
    dayofmonth("event_time").alias("day"),
    quarter("event_time").alias("quarter"),
    dayofweek("event_time").alias("day_of_week"),
    date_format(col("date_id"), "EEEE").alias("day_name")
).distinct()

# dim_event_type
dim_event_type = spark.createDataFrame([
    ("view", 1),
    ("cart", 2),
    ("purchase", 3)
], ["event_type_name", "event_type_id"])

# dim_product
dim_product = df_cleaned.select(
    "product_id", "brand", "category_id", col("price").alias("price_current")
).distinct()

# dim_category
dim_category = df_cleaned.select(
    "category_id", "category_level_1", "category_level_2", "category_level_3", "category_level_4"
).distinct()

In [29]:
dim_date.show()

+----------+-----+---+-------+-----------+---------+
|   date_id|month|day|quarter|day_of_week| day_name|
+----------+-----+---+-------+-----------+---------+
|2020-01-01|    1|  1|      1|          4|Wednesday|
|2020-01-02|    1|  2|      1|          5| Thursday|
|2020-01-03|    1|  3|      1|          6|   Friday|
|2020-01-04|    1|  4|      1|          7| Saturday|
|2020-01-05|    1|  5|      1|          1|   Sunday|
|2020-01-06|    1|  6|      1|          2|   Monday|
|2020-01-07|    1|  7|      1|          3|  Tuesday|
|2020-01-08|    1|  8|      1|          4|Wednesday|
|2020-01-09|    1|  9|      1|          5| Thursday|
|2020-01-10|    1| 10|      1|          6|   Friday|
|2020-01-11|    1| 11|      1|          7| Saturday|
|2020-01-12|    1| 12|      1|          1|   Sunday|
|2020-01-13|    1| 13|      1|          2|   Monday|
|2020-01-14|    1| 14|      1|          3|  Tuesday|
|2020-01-15|    1| 15|      1|          4|Wednesday|
|2020-01-16|    1| 16|      1|          5| Thu

In [30]:
df_count_event_by_product = df_cleaned.groupBy("product_id", "event_type").count() \
        .groupBy("product_id") \
        .pivot("event_type", ["view", "cart", "purchase"]) \
        .sum("count") \
        .fillna(0)

df_count_event_by_product = df_count_event_by_product.withColumn("total_events", col("view") + col("cart") + col("purchase"))\
                                                     .withColumn("purchase_conversion", round((col("purchase") / (col("view") + col("cart"))) * 100, 2))\
                                                     .sort("total_events", ascending=False)

In [31]:
df_count_event_by_product.show()

+----------+------+-----+--------+------------+-------------------+
|product_id|  view| cart|purchase|total_events|purchase_conversion|
+----------+------+-----+--------+------------+-------------------+
|   1004767|768721|92406|   36854|      897981|               4.28|
|   1005115|522888|64899|   26401|      614188|               4.49|
|   1005160|454484|38689|   11894|      505067|               2.41|
|   1005100|389261|61243|   26996|      477500|               5.99|
|   4804056|256735|47650|   15442|      319827|               5.07|
|   1002544|257457|35616|   14992|      308065|               5.12|
|   1004873|268510|23133|    9795|      301438|               3.36|
|   1005212|239200|39409|   16553|      295162|               5.94|
|   1005105|250436|25733|   10629|      286798|               3.85|
|   1005174|240034|11977|    5475|      257486|               2.17|
|   1004856|217525|23721|    9008|      250254|               3.73|
|   4804718|218969|10470|    3386|      232825| 

In [ ]:
# # === FACT TABLE: fact_summary ===
# fact = fact_events \
#     .join(dim_product, "product_id", "left") \
#     .join(dim_category, "category_id", "left") \
#     .join(dim_time.select("time_id", to_date("event_time").alias("date"), "hour"), "time_id", "left")

# fact_summary = fact.groupBy("date", "hour", "brand", "category_level_1").agg(
#     count(when(col("event_type_id") == 1, True)).alias("view_count"),
#     count(when(col("event_type_id") == 2, True)).alias("cart_count"),
#     count(when(col("event_type_id") == 3, True)).alias("purchase_count"),
#     sum(when(col("event_type_id") == 3, col("price"))).alias("revenue")
# ).withColumn("conversion_rate_view_to_purchase",
#              round(100.0 * col("purchase_count") / when(col("view_count") > 0, col("view_count")), 2)) \
#  .withColumn("conversion_rate_cart_to_purchase",
#              round(100.0 * col("purchase_count") / when(col("cart_count") > 0, col("cart_count")), 2))

In [ ]:
df_1 = df.groupBy('category_id','event_type').agg(count('*').alias('counts'))

In [ ]:
df_1.groupBy("category_id").pivot("event_type", ["view", "cart"]).sum("counts").show()

In [ ]:
df.groupBy("brand").count().orderBy("count", ascending=False).show()

In [ ]:
spark.stop()